# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START","PAD_START","1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=1, dealer=2, ddsolver=dds, bba_is_controlling=False, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '3D',
  'insta_score': 0.559,
  'adjustment': 1.34,
  'expected_score': -36,
  'expected_imp': 1.98,
  'expected_tricks': 8.7,
  'alert': 'False',
  'explanation': 'Bidable suit -- 6-!C; 5+!D; 7-11 HCP'},
 {'call': 'PASS',
  'insta_score': 0.521,
  'adjustment': 1.25,
  'expected_score': -124,
  'expected_imp': -1.98,
  'expected_tricks': 8.6}]

In [3]:
bid.samples

['Axxxxxx..QJT.Jxx J.Jxx.Axxxxx.Txx Tx.KTxxxx.Kx.KQx KQx.AQxx.xx.Axxx - 0.94676 | 1H-1N-2S-P-P-P (9) | 1H-1N-2S-3D-P-P-P (8)',
 'Qxxxxx.Kx.QJT.xx J.Jxx.Axxxxx.Txx KTx.Txxxx.x.AKJx Axx.AQx.Kxx.Qxxx - 0.94462 | 1H-1N-2S-P-P-P (9) | 1H-1N-2S-3D-3S-P-P-4D-P-P-4S-P-P-P (9)',
 'AQxxxx.Q.Txx.xxx J.Jxx.Axxxxx.Txx xxx.Kxxxx.KJ.Axx KTx.ATxx.Qx.KQJx - 0.94411 | 1H-1N-2S-P-P-P (8) | 1H-1N-2S-3D-3S-P-4S-P-P-P (8)',
 'KQTxxxx..Jxx.Jxx J.Jxx.Axxxxx.Txx xx.KQxxxx.Tx.AQx Axx.ATxx.KQ.Kxxx - 0.94321 | 1H-1N-2S-P-P-P (8) | 1H-1N-2S-3D-P-P-P (9)',
 'KTxxxxx.x.xx.Axx J.Jxx.Axxxxx.Txx xx.AQxxx.QT.KJxx AQx.KTxx.KJx.Qxx - 0.93697 | 1H-1N-2S-P-P-P (9) | 1H-1N-2S-3D-P-P-P (7)',
 'Qxxxxx.x.QTx.Axx J.Jxx.Axxxxx.Txx Axx.ATxxxx.x.QJx KTx.KQx.KJx.Kxxx - 0.93588 | 1H-1N-2S-P-3S-P-P-4D-P-P-4S-P-P-P (10) | 1H-1N-2S-3D-3S-P-P-4D-P-P-4S-P-P-5D-X-P-P-P (7)',
 'KTxxxx.x.JTx.Axx J.Jxx.Axxxxx.Txx Qxx.AKQTxx.x.xxx Axx.xxx.KQx.KQJx - 0.93461 | 1H-1N-2S-P-P-P (11) | 1H-1N-2S-3D-3S-P-4S-P-P-P (11)',
 'KTxxxxx.x.JTx.Kx J.Jxx.Axxxx

In [4]:
auction = ["PAD_START","PAD_START","1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 1.08,
  'adjustment': 0,
  'expected_score': -157,
  'expected_tricks': 9.4,
  'alert': 'False',
  'explanation': ' -- 6-!C; 5+!D; 7-8 HCP'},
 {'call': '5D', 'insta_score': 0.113}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AKxxx.xx.xx.Jxxx J.Jxx.Axxxxx.Txx Qxxx.ATxxx.Jx.Ax Txx.KQx.KQT.KQxx - 0.71845',
 'AQTxx.x.QJTx.xxx J.Jxx.Axxxxx.Txx xxxx.AKTxx.x.Kxx Kxx.Qxxx.Kx.AQJx - 0.71404',
 'AKTxx.xx.T.xxxxx J.Jxx.Axxxxx.Txx xxxx.AQTxx.QJx.K Qxx.Kxx.Kxx.AQJx - 0.70547',
 'Axxxx.AT.Tx.Jxxx J.Jxx.Axxxxx.Txx Txxx.Kxxxx.K.Axx KQx.Qxx.QJxx.KQx - 0.70453',
 'QTxxxx.xx.Jxx.Ax J.Jxx.Axxxxx.Txx Axxx.AQxxx..Qxxx Kx.KTx.KQTx.KJxx - 0.70335',
 'Txxxx.Ax.xx.Axxx J.Jxx.Axxxxx.Txx AKxx.Qxxxx.J.Jxx Qxx.KTx.KQTx.KQx - 0.70279',
 'QTxxxx.x.Jx.AQxx J.Jxx.Axxxxx.Txx Axxx.AQxxx.x.xxx Kx.KTxx.KQTx.KJx - 0.69771',
 'AQxxx.xx.Jxx.xxx J.Jxx.Axxxxx.Txx KTxx.AQxxx.K.xxx xxx.KTx.QTx.AKQJ - 0.67814',
 'KQTxxx.xx.T.Qxxx J.Jxx.Axxxxx.Txx Axxx.AKTxx.Jx.xx xx.Qxx.KQxx.AKJx - 0.67648',
 'QTxxx.x.KTx.Jxxx J.Jxx.Axxxxx.Txx Axxx.Axxxx.Jx.Ax Kxx.KQTx.Qx.KQxx - 0.67592',
 'Qxxxx.x.KQx.xxxx J.Jxx.Axxxxx.Txx ATxx.AQTxx.xx.Qx Kxx.Kxxx.JT.AKJx - 0.67376',
 'AKxxxx.xx.Tx.Jxx J.Jxx.Axxxxx.Txx Qxxx.KQTxx.x.Axx Tx.Axx.KQJx.KQxx - 0.67207',
 'Kxxxx.Qx.KJTx.